Facial recognization using opencv

In [1]:
import os
import cv2
import shutil

# Load the Haar cascade classifiers for face detection
cascade_files = ['face_model/haarcascade_frontalface_alt.xml',
                 'face_model/haarcascade_frontalface_alt.xml',
                 'face_model/haarcascade_profileface.xml']

face_cascades = [cv2.CascadeClassifier(file) for file in cascade_files]

# Input and output directories
input_dir = 'test'
output_dir = 'test_cropped'

# Define the target size for the extracted faces
target_size = (224, 224)  # Specify width and height in pixels

# Iterate over each image in the input directory
for filename in os.listdir(input_dir):
    try:
        # Read the input image
        img = cv2.imread(os.path.join(input_dir, filename))

        # If the image cannot be read, skip to the next image
        if img is None:
            shutil.copy(os.path.join(input_dir, filename), os.path.join(output_dir, filename))
            continue

        # Convert into grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        faces_detected = False

        # Iterate over each cascade classifier
        for face_cascade in face_cascades:
            # Detect faces using the current classifier
            faces = face_cascade.detectMultiScale(gray, 1.1, 10)

            # If faces are detected, set the flag to True and break out of the loop
            if len(faces) > 0:
                faces_detected = True
                break

        # If faces are detected using any of the classifiers
        if faces_detected:
            # Iterate over each detected face
            for i, (x, y, w, h) in enumerate(faces):
                # Extract the region of interest (face) from the image and resize it to the target size
                face_roi = cv2.resize(img[y:y+h, x:x+w], target_size)

                # Save the recognized face as a separate JPEG file
                output_filename = os.path.splitext(filename)[0] + f'.jpg'
                output_path = os.path.join(output_dir, output_filename)
                cv2.imwrite(output_path, face_roi)

                # Draw rectangle around the detected face on the original image
                # cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

        else:  # If no faces are detected using any of the classifiers
            # Resize the image to the target size and save it under the same name as before
            resized_img = cv2.resize(img, target_size)
            cv2.imwrite(os.path.join(output_dir, filename), resized_img)
    except Exception as e:
        print(f"Error processing image {filename}: {e}")

